In [1]:
import io
import re
import csv
import pandas as pd
import win32com.client
from docx import Document
import os

In [2]:
prov = open('../provinsi.txt', 'r').read().lower()

In [3]:
prov = prov.split('\n')

In [4]:
prov = set(prov[:len(prov)-1])

In [5]:
def extract_table(doc, prov=prov, kategori=''):
    i = 0
    y = 0
    par2 = []
    kab = []
    kab_bold = []
    par_1 = doc.paragraphs
    for idx, para in enumerate(par_1):
        if idx in kab_bold:
            continue
        temp = {}
        if idx == len(par_1)-1:
            if len(kab) == 0:
                continue
            else:
                kab[i-1]['end_pos'] = idx
        for r in para.runs:
            if r.font.size == 114300:
                temp_kabkot = para.text
                cek_tit = True
                xx = idx + 1
                if(xx == len(par_1)):
                    cek_tit = False
                while cek_tit:
                    temp_kabkot2 = [run.text for run in par_1[xx].runs if (run.bold) and (run.font.size == 114300)]
#                     print(temp_kabkot2)
                    if len(temp_kabkot2) > 0:
                        kab_bold.append(xx)
                        temp_kabkot = temp_kabkot +' '+temp_kabkot2[0]
                        xx = xx + 1
                    elif(xx == len(par_1)):
                        cek_tit = False
                    else:
                        cek_tit = False
                temp['text'] = temp_kabkot
                if (i != 0) and (idx != len(par_1)-1):
                    kab[i-1]['end_pos'] = idx-2
                kab.append(temp)
                i = i+1
            else:
                temp2 = y, para
                par2.append(temp2)
                y = y+1
     #---------------------------------------------------------------------------------------------------------------------#            
    i = 0
    kab_temp = kab
    kab = []
    # par_1 = []
    for k in kab_temp:
        if (k['text'].lower() not in prov) and (len(k['text'].strip(' \n\r\t')) != 0):
            kab.append(k)
    kab = pd.DataFrame(kab)        
    print(kab)
    par = par2
    for idx, para in par:
        for run in para.runs:
            if (run.font.size == 101600) and (run.bold):
                run.bold = None
    #---------------------------------------------------------------------------------------------------------------------#
    batas = 0
    pars = []
    title_bold = []
    kat_bold = []
    for idx, para in par:
        if (idx in title_bold) :
            continue
        paratemp = []
        for run in para.runs:
            if run.bold:
                temptit = [run.text for run in para.runs if run.bold][0]
                cek_tit = True
                xx = idx + 1
                if(xx == len(par)):
                    cek_tit = False
                while cek_tit:
                    temptit2 = [run.text for run in par[xx][1].runs if run.bold]
                    if len(temptit2) > 0:
                        title_bold.append(xx)
                        temptit = temptit +' '+temptit2[0]
                        xx = xx + 1
                        if(xx == len(par)):
                            cek_tit = False
                    else:
                        cek_tit = False
                paratemp.append(temptit+' [NAMA_PERUSAHAAN]')
                for x in range(batas, idx):
                    if x in kat_bold:
                        continue
                    for run in par[x][1].runs:
                        if len(run.text.strip(' \n\r\t')) != 0:
                            if run.bold:
                                continue
                            elif run.font.size == 101600:
                                temp_kat = run.text
                                xxx = x + 1
                                cek_kat = True
                                if xxx < batas:
                                    cek_kat = False
                                while cek_kat:
                                    for run in par[xxx][1].runs:
                                        if (run.font.size == 101600):
                                            temp_kat = temp_kat +' '+par[xxx][1].text
                                            kat_bold.append(xxx)
                                            xxx = xxx + 1
                                            if xx >= batas:
                                                cek_kat = False
                                        else:
                                            cek_kat = False
                                paratemp.append(temp_kat)
                            else:
                                temp_alamat1 = par[x][1].text
                                paratemp.append(temp_alamat1)

                pars.append(paratemp)
                batas = xx
                
     #---------------------------------------------------------------------------------------------------------------------#
    # menggabungkan alamat
    par_temp = []
    for idx, p in enumerate(pars):
        parss = []
        batas = len(p)-1
        batas_bawah = []
        for idxx, x in enumerate(p):
            if idxx in batas_bawah:
                continue
            tempAlamat = ''
            if idxx <= batas:
                if re.search(r'( \[NAMA_PERUSAHAAN\])', x):
                    parss.append(x)
                elif (re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', x)):
                    if idxx == len(p)-1:
                        tempAlamat = x
                    else:
                        xxx = idxx
                        while (xxx <= len(p)-1):
                            if (re.search(r'[^\-]\b[\d]{5}\b', p[xxx])):
                                batas_bawah.append(xxx)
                                tempAlamat = tempAlamat + ' '+ p[xxx]
                                xxx = xxx + 1
                            elif (re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', x)):
                                batas_bawah.append(xxx)
                                tempAlamat = tempAlamat + ' '+ p[xxx]
                                xxx = xxx + 1
                    if len(kab) > 0:
                        if (re.search(r'[^\-]\b[\d]{5}\b', tempAlamat)) and (re.search(r'Ext: \b[\d]{5}\b', tempAlamat) == None):
                            kode_pos = re.search(r'[^\-]\b[\d]{5}\b', tempAlamat).group().strip()
                            alamat = re.sub(r'[^\-]\b[\d]{5}\b', '',  tempAlamat)
                            temp_alamat1 = alamat + ', '+ kab[kab.end_pos >= idx].head(1).iloc[0].text + ' '+kode_pos
                        else:
                            if (re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', tempAlamat)):
                                temp_alamat1 = tempAlamat + ', '+ kab[kab.end_pos >= idx].head(1).iloc[0].text
                            else:
                                temp_alamat1 = tempAlamat
                    else:
                        temp_alamat1 = tempAlamat
                    parss.append(temp_alamat1.strip(' \n\r\t'))
                else:
                    parss.append(x)
        par_temp.append(parss)
    #---------------------------------------------------------------------------------------------------------------------#    
    # Buat DF
    data = []
    tempTel1 = ''
    tempTel2 = ''
    tempTel3 = ''  
    tempEmail = ''
    tempWeb = ''
    for line in par_temp:

        row = {'Kategori': kategori,
               'Telp1': '', 
               'Telp2': '', 
               'Telp3': '',
               'Email': '',
               'Website': '',
               'Alamat': '',
               'Nama Perusahaan': '',
               'Kode Kategori': '',
               'Status Badan Usaha': ''}

        for text in line:
            if(re.search(r'[0-9]{3,}-\d{5,}|^08|\+62|\d{3}-62-\d{2,3}-\d{5,}|--\d{5,}|\d{6,}', text)):
                if(tempTel1 == ''):
                    tempTel1 = text
                    row['Telp1'] = tempTel1
                elif(tempTel2 == ''):
                    tempTel2 = text
                    row['Telp2'] = tempTel2
                else:
                    tempTel3 = text
                    row['Telp3'] = tempTel3
            elif(re.search(r'( \[NAMA_PERUSAHAAN\])', text)):
                text = re.sub(r'( \[NAMA_PERUSAHAAN\])', '', text)
                if (re.search(r'(, ?\bPD\b.*)|(, ?\bPT\b.*)|(, ?\bCO\b.?)|(, ?\bCV\b.?)|(,? ?\bBUT\b)|(,? ?\bGMBH\b)|(,? ?\bLTD.*\b)|(,? ?\bPTE\b)|(, ?\bINC\b)|(,? ?\bS\.R\.L\b)|(,? ?\bS\.P\.A\b)|(,? ?\bSDN BHD\b)', text)):
                    row['Nama Perusahaan'] = re.sub(r'(, ?\bPD\b.*)|(, ?\bPT\b.*)|(, ?\bCO\b.?)|(, ?\bCV\b.?)|(,? ?\bBUT\b)|(,? ?\bGMBH\b)|(,? ?\bLTD.*\b)|(,? ?\bPTE\b)|(, ?\bINC\b)|(,? ?\bS\.R\.L\b)|(,? ?\bS\.P\.A\b)|(,? ?\bSDN BHD\b)','',text)
                    row['Status Badan Usaha'] = re.search(r'(, ?\bPD\b.*)|(, ?\bPT\b)|(, ?\bCO\b.?)|(, ?\bCV\b.?)|(,? ?\bBUT\b)|(,? ?\bGMBH\b)|(,? ?\bLTD\b)|(,? ?\bPTE\b)|(, ?\bINC\b)|(,? ?\bS\.R\.L\b)|(,? ?\bS\.P\.A\b)|(,? ?\bSDN BHD\b)', text).group().replace(',','').strip(' \n\r\t')
                else:
                    row['Nama Perusahaan'] = text
                    if(re.search(r'(\bS\.R\.L\b)|(\bSDN BHD\b)|(\bLTD\b)|(\bGMBH\b)|(\bBUT\b)|(\bS\.P\.A\b)',text)):
                        row['Status Badan Usaha'] = re.search(r'(\bS\.R\.L\b)|(\bSDN BHD\b)|(\bLTD\b)|(\bBUT\b)|(\bGMBH\b)|(\bS\.P\.A\b)',text).group()
                    elif(re.search(r'\bLIMITED\b',text)):
                        row['Status Badan Usaha'] = 'LTD'
                    elif(re.search(r'\b KAB \b|\b KOTA \b',text)):
                        row['Status Badan Usaha'] = 'PD'
                    else:
                        row['Status Badan Usaha'] = 'None'
            elif(re.search(r'.*@.*\.[a-zA-Z]{2,3}\.?', text)) or (re.search(r'.*@.*', text)):
                tempEmail = text
                row['Email'] = tempEmail
            elif(re.search(r'(^www.)|(^http)|(.com$)|(.co.id$)|(.net$)', text)):
                tempWeb = text
                row['Website'] = tempWeb
            elif(re.search(r'^[0-9]{2}[a-zA-Z\s\-]+( - .*)?$', text)):
                row['Kategori'] = re.sub(r'\d+ | - .*','', text)
                row['Kode Kategori'] = re.sub(r' [a-zA-Z].*','', text)
            elif(re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', text)):
                row['Alamat'] = (row['Alamat']+' '+text).strip(' \n\r\t')
            else:
                row['Alamat'] = (row['Alamat']+' '+text).strip(' \n\r\t')
        data.append(row)
        tempTel1 = ''
        tempTel2 = ''
        tempTel3 = ''  
        tempEmail = ''
        tempWeb = ''
        
    return pd.DataFrame(data)

In [6]:
def list_files(basePath, validExts=(".docx", ".doc"), contains=None):
    # loop over the directory structure
    for (rootDir, dirNames, filenames) in os.walk(basePath):
        # loop over the filenames in the current directory
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
                continue

            # determine the file extension of the current file
            ext = filename[filename.rfind("."):].lower()

            # check to see if the file is an image and should be processed
            if ext.endswith(validExts):
                # construct the path to the image and yield it
                filePath = filename
                yield filePath

In [7]:
list_docx = list(list_files("D:\\BPS\\pdfScrapper\\L"))

In [8]:
for path in list_docx:
    doc = Document(open(path, 'rb'))
    var_path = path.lower().replace(' ', '_').replace('.docx', '').replace(',', '')
    kat = path.replace('.docx', '')
    print(var_path)
    print(kat)
    exec(var_path +' = extract_table(doc,prov,"'+kat+'")')
    exec(var_path+'.to_csv("'+var_path+'.csv")')

real_estat
Real Estat
     end_pos                     text
0          3        KAB. ACEH SELATAN
1         15       KAB. ACEH TENGGARA
2         19          KAB. ACEH TIMUR
3         25         KAB. ACEH TENGAH
4         30          KAB. ACEH BARAT
5         48          KAB. ACEH BESAR
6         53          KAB. ACEH UTARA
7         61     KAB. ACEH BARAT DAYA
8         79        KAB. ACEH TAMIANG
9         86        KAB. BENER MERIAH
10       134          KOTA BANDA ACEH
11       153              KOTA LANGSA
12       160         KOTA LHOKSEUMAWE
13       168     KAB. TAPANULI TENGAH
14       173        KAB. LABUHAN BATU
15       182              KAB. ASAHAN
16       186          KAB. SIMALUNGUN
17       237        KAB. DELI SERDANG
18       241       KAB. PAKPAK BHARAT
19       249           KAB. BATU BARA
20       254  KAB. PADANG LAWAS UTARA
21       272    KOTA PEMATANG SIANTAR
22       276       KOTA TEBING TINGGI
23       732               KOTA MEDAN
24       741              KO